In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install -q evaluate transformers datasets rouge_score jiwer
!pip install -q git+https://github.com/google-research/bleurt.git
!pip install -q nltk textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSeq2SeqLM, pipeline, AutoTokenizer, AutoModelForSequenceClassification
import nltk
nltk.download('punkt')
import textstat
import torch
import pprint
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
#def calculate_flesch_kincaid(text: list[str]):
#  """Calculates the flesch kincaid score for a list of text"""
#  read_obj = [Readability(t) for t in text]
#  return [r.flesch_kincaid().score for r in read_obj]

entailment_tokenizer = AutoTokenizer.from_pretrained("khalidalt/DeBERTa-v3-large-mnli")
entailment_model = AutoModelForSequenceClassification.from_pretrained("khalidalt/DeBERTa-v3-large-mnli")

def calculate_entailment(
    premise,
    hypothesis,
    device,
    label_names = ["entailment", "neutral", "contradiction"]
):
  input = entailment_tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
  input.to(device)
  entailment_model.to(device)
  output = entailment_model(input["input_ids"])  # device = "cuda:0" or "cpu"

  prediction = torch.softmax(output["logits"][0], -1)

  return label_names[prediction.argmax(0).tolist()]

def encode_decode(df):
    input = "summarize: " + df[input_col]
    inputs = tokenizer(input, return_tensors = "pt", max_length = max_input_len, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length = 128)
    result = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {summary_col: result}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
METRIC_1 = "rouge"
METRIC_2 = "bleurt"
rouge = evaluate.load("rouge")
bleurt = evaluate.load("bleurt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [5]:
  dataset = load_dataset("jordanfan/processed_us_congress_117_bills_v3")
  dataset_test = dataset['test']
  cols_to_keep = ["index", "policy_areas", "cur_text", "cleaned_summary",
                               'extracted_text', 'extracted_text_375', 'extracted_text_750', 'extracted_text_1000',
                               'bertsum_extracted_250', 'bertsum_extracted_375',
                               'bertsum_extracted_375_1000', 'bertsum_extracted_250_1000', 'bertsum_extracted_375_750', 'bertsum_extracted_250_750',
                              'bertsum_extracted_375_500', 'bertsum_extracted_250_500', 'bertsum_extracted_375_375', 'bertsum_extracted_250_375']
  cols_to_remove = [col for col in dataset_test.column_names if col not in cols_to_keep]
  dataset_test = dataset_test.remove_columns(cols_to_remove)

Generating train split:   0%|          | 0/11277 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3388 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/377 [00:00<?, ? examples/s]

# T5 - Untrained


In [6]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
max_input_len = 512
input_col = "cur_text"
summary_col = "summary_untrained"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/eb2a76efb95e30793977987dd59bdd9acc0f50ae', commit_message='Upload dataset', commit_description='', oid='eb2a76efb95e30793977987dd59bdd9acc0f50ae', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_untrained"]))
bleurt_untrained = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_untrained"])["scores"]
print(np.mean(bleurt_untrained))
print(np.median(bleurt_untrained))
print(np.std(bleurt_untrained))

{'rouge1': 0.2692962222945592, 'rouge2': 0.10340977805924306, 'rougeL': 0.20039906689594203, 'rougeLsum': 0.19987306246656153}
-0.7487824393796984
-0.7414690256118774
0.32637714722482525


In [8]:
readability_untrained = pd.Series(dataset_test["summary_untrained"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_untrained))
print(np.median(readability_untrained))
print(np.std(readability_untrained))

47.15244031830239
49.45
19.006405101797995


In [9]:
pd.DataFrame(dataset_test)[["cleaned_summary", "summary_untrained"]].apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_untrained"], device), axis = 1).value_counts(normalize = True)

neutral          0.745358
entailment       0.214854
contradiction    0.039788
Name: proportion, dtype: float64

# T5 - Baseline

In [10]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-baseline")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-baseline")
max_input_len = 512
input_col = "cur_text"
summary_col = "summary_baseline_512"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/90d3113a3ce30f54de56c1f5e365fc64038f57a4', commit_message='Upload dataset', commit_description='', oid='90d3113a3ce30f54de56c1f5e365fc64038f57a4', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_baseline_512"]))
bleurt_baseline = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_baseline_512"])["scores"]
print(np.mean(bleurt_baseline))
print(np.median(bleurt_baseline))
print(np.std(bleurt_baseline))

{'rouge1': 0.43204031868104464, 'rouge2': 0.2565310452353503, 'rougeL': 0.3541599241615871, 'rougeLsum': 0.35438373125166445}
-0.2348145818540367
-0.24938486516475677
0.3283944749702856


In [12]:
readability_baseline_512 = pd.Series(dataset_test["summary_baseline_512"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_baseline_512))
print(np.median(readability_baseline_512))
print(np.std(readability_baseline_512))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_baseline_512"], device), axis = 1).value_counts(normalize = True))

26.6163925729443
27.83
21.65349597137206
neutral          0.554377
entailment       0.384615
contradiction    0.061008
Name: proportion, dtype: float64


# T5 - Extractive 375



In [13]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-extractive-375")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-extractive-375")
max_input_len = 512
input_col = "extracted_text_375"
summary_col = "summary_extractive_512_375"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/7e5c4c887046e03826ad6700499b976175f510d5', commit_message='Upload dataset', commit_description='', oid='7e5c4c887046e03826ad6700499b976175f510d5', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_extractive_512_375"]))
bleurt_extractive_375 = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_extractive_512_375"])["scores"]
print(np.mean(bleurt_extractive_375))
print(np.median(bleurt_extractive_375))
print(np.std(bleurt_extractive_375))

{'rouge1': 0.40406929551507775, 'rouge2': 0.229581665795422, 'rougeL': 0.33102417964912967, 'rougeLsum': 0.33087064946675804}
-0.29493807109976955
-0.30967384576797485
0.34009752573556506


In [15]:
readability_extractive_512_375 = pd.Series(dataset_test["summary_extractive_512_375"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_375))
print(np.median(readability_extractive_512_375))
print(np.std(readability_extractive_512_375))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_512_375"], device), axis = 1).value_counts(normalize = True))

30.44031830238727
31.21
19.843113255365495
neutral          0.538462
entailment       0.421751
contradiction    0.039788
Name: proportion, dtype: float64


# T5 - Extractive 500

In [16]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-extractive-500")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-extractive-500")
max_input_len = 512
input_col = "extracted_text"
summary_col = "summary_extractive_512_500"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/016326f380964ba666d6f4ac9de4519bb473e97d', commit_message='Upload dataset', commit_description='', oid='016326f380964ba666d6f4ac9de4519bb473e97d', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_extractive_512_500"]))
bleurt_extractive_500 = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_extractive_512_500"])["scores"]
print(np.mean(bleurt_extractive_500))
print(np.median(bleurt_extractive_500))
print(np.std(bleurt_extractive_500))

{'rouge1': 0.41559170628488284, 'rouge2': 0.23628299724826057, 'rougeL': 0.3366000808606124, 'rougeLsum': 0.3363018339200696}
-0.2766710791825774
-0.3009733259677887
0.3385719670275896


In [18]:
readability_extractive_512_500 = pd.Series(dataset_test["summary_extractive_512_500"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_500))
print(np.median(readability_extractive_512_500))
print(np.std(readability_extractive_512_500))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_512_500"], device), axis = 1).value_counts(normalize = True))

27.761830238726795
28.17
20.245407052707083
neutral          0.538462
entailment       0.405836
contradiction    0.055703
Name: proportion, dtype: float64


# T5 - BERTSUM - 375





In [19]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-bertsum-375")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-bertsum-375")
max_input_len = 512
input_col = "bertsum_extracted_250_375"
summary_col = "summary_bertsum_extractive_512_375"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/049443d665625d0606e96c952a47b6f484c373de', commit_message='Upload dataset', commit_description='', oid='049443d665625d0606e96c952a47b6f484c373de', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_bertsum_extractive_512_375"]))
bleurt_bertsum_375 = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_bertsum_extractive_512_375"])["scores"]
print(np.mean(bleurt_bertsum_375))
print(np.median(bleurt_bertsum_375))
print(np.std(bleurt_bertsum_375))

{'rouge1': 0.413532601769633, 'rouge2': 0.23339872808985315, 'rougeL': 0.33844948098544314, 'rougeLsum': 0.3385950015171919}
-0.2559657519292926
-0.2731882333755493
0.32121369211753126


In [21]:
readability_extractive_512_375 = pd.Series(dataset_test["summary_bertsum_extractive_512_375"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_375))
print(np.median(readability_extractive_512_375))
print(np.std(readability_extractive_512_375))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_bertsum_extractive_512_375"], device), axis = 1).value_counts(normalize = True))

30.476153846153846
30.2
19.014711024354977
neutral          0.533156
entailment       0.421751
contradiction    0.045093
Name: proportion, dtype: float64


# T5 - BERTSUM - 500

In [22]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-bertsum-500")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-bertsum-500")
max_input_len = 512
input_col = "bertsum_extracted_250_500"
summary_col = "summary_bertsum_extractive_512_500"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries/commit/7fc5abe63a1a6f888472e5532da4d2e04132d3b7', commit_message='Upload dataset', commit_description='', oid='7fc5abe63a1a6f888472e5532da4d2e04132d3b7', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
print(rouge.compute(references = dataset_test["cleaned_summary"],
              predictions = dataset_test["summary_bertsum_extractive_512_500"]))
bleurt_extractive_500 = bleurt.compute(references = dataset_test["cleaned_summary"],
                                predictions = dataset_test["summary_bertsum_extractive_512_500"])["scores"]
print(np.mean(bleurt_extractive_500))
print(np.median(bleurt_extractive_500))
print(np.std(bleurt_extractive_500))

{'rouge1': 0.4126778339188786, 'rouge2': 0.23170131048844722, 'rougeL': 0.33882006620527866, 'rougeLsum': 0.3384160721215994}
-0.26182232586671883
-0.2741042375564575
0.3239889565650425


In [24]:
readability_extractive_512_500 = pd.Series(dataset_test["summary_bertsum_extractive_512_500"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_500))
print(np.median(readability_extractive_512_500))
print(np.std(readability_extractive_512_500))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_bertsum_extractive_512_500"], device), axis = 1).value_counts(normalize = True))

30.03153846153846
30.5
18.41734098864497
neutral          0.527851
entailment       0.416446
contradiction    0.055703
Name: proportion, dtype: float64


# T5 - ABS 2 ABS

In [25]:
dataset_billsum = load_dataset("jordanfan/billsum_abstracted_us_congress_117_bills_all")
dataset_billsum_test = dataset_billsum['test']
dataset_billsum_test

Generating train split:   0%|          | 0/10988 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3310 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/367 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'index', 'id', 'policy_areas', 'cur_summary', 'cur_text', 'title', 'titles_official', 'titles_short', 'sponsor_name', 'sponsor_party', 'sponsor_state', 'cleaned_summary', 'extracted_text', 'extracted_text_375', 'extracted_text_750', 'extracted_text_1000', 'bertsum_extracted_250', 'bertsum_extracted_375', 'bertsum_extracted_375_1000', 'bertsum_extracted_250_1000', 'bertsum_extracted_375_750', 'bertsum_extracted_250_750', 'bertsum_extracted_375_500', 'bertsum_extracted_250_500', 'bertsum_extracted_375_375', 'bertsum_extracted_250_375', 'text_len', 'billsum_abstracted_1000', 'billsum_abstracted_500', '__index_level_0__'],
    num_rows: 367
})

In [26]:
tokenizer = AutoTokenizer.from_pretrained("jgibb/t-5-base-abs2abs")
model = AutoModelForSeq2SeqLM.from_pretrained("jgibb/t-5-base-abs2abs")
max_input_len = 512
input_col = "billsum_abstracted_500"
summary_col = "summary_billsum_abstracted_500"
dataset_billsum_test = dataset_billsum_test.map(encode_decode)
dataset_billsum_test.push_to_hub("jgibb/congress_117_bills_test_t5_summaries_billsum")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jgibb/congress_117_bills_test_t5_summaries_billsum/commit/8ccb94a96beb937aaeadd24340c537dd90a53862', commit_message='Upload dataset', commit_description='', oid='8ccb94a96beb937aaeadd24340c537dd90a53862', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
print(rouge.compute(references = dataset_billsum_test["cleaned_summary"],
              predictions = dataset_billsum_test["summary_billsum_abstracted_500"]))
bleurt_abstractive_500 = bleurt.compute(references = dataset_billsum_test["cleaned_summary"],
                                predictions = dataset_billsum_test["summary_billsum_abstracted_500"])["scores"]
print(np.mean(bleurt_abstractive_500))
print(np.median(bleurt_abstractive_500))
print(np.std(bleurt_abstractive_500))

{'rouge1': 0.4268537786314913, 'rouge2': 0.24970760863534747, 'rougeL': 0.3512435206840042, 'rougeLsum': 0.3521624320795338}
-0.2548356342055817
-0.2613086998462677
0.30285265012894647


In [28]:
readability_abstractive_500 = pd.Series(dataset_billsum_test["summary_billsum_abstracted_500"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_abstractive_500))
print(np.median(readability_abstractive_500))
print(np.std(readability_abstractive_500))
print(pd.DataFrame(dataset_billsum_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_billsum_abstracted_500"], device), axis = 1).value_counts(normalize = True))

28.719346049046322
28.17
19.849943619509066
neutral          0.580381
entailment       0.376022
contradiction    0.043597
Name: proportion, dtype: float64
